In [ ]:
##importing regular expression
import re
import pandas as pd

In [ ]:
f = open('Enter your text file path','r',encoding='utf-8')

In [ ]:
data = f.read()

In [ ]:
print(data)

In [ ]:
print(type(data))

In [ ]:
##making the pattern out of date and time
# pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s?[APMampm]{0,2}\s-\s|\[\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}:\d{2}\s?[APMampm]{0,2}\]|[0-9]{2}/[0-9]{2}/[0-9]{2,4},\s\d{1,2}:\d{2}\s?[APMampm]{0,2}|\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}'


In [ ]:
# splitting the data into messages and date & time
messages = re.split(pattern,data)[1:]
messages

In [ ]:
len(messages)

In [ ]:
dates = re.findall(pattern,data)
dates
len(dates)

In [ ]:
# from datetime import datetime
# # dividing the dataset into two columns messages and date 
import pandas as pd
from dateutil import parser
df = pd.DataFrame({'user_message':messages,'message_date':dates})


def parse_date(date_str):
    try:
        # Attempt to parse date assuming it's in a recognized format
        return parser.parse(date_str.split(' - ')[0])
    except (parser.ParserError, ValueError):
        raise ValueError(f"Date format not recognized: {date_str}")

# Assuming df is your DataFrame
df['message_date'] = df['message_date'].apply(parse_date)
df.rename(columns={'message_date':'date'}, inplace=True)

# Now the DataFrame should be correctly formatted
df.head()



In [ ]:
df.shape

In [ ]:
# separate users and messages 
users = []
messages = []
for message in df['user_message']:
    entry = re.split(r'([\w\W]*?):\s', message)
    if entry[1:]: # user name
        users.append(entry[1])
        messages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['users'] = users
df['message'] = messages
df1 = df.drop(columns=['user_message'],inplace=True)

df.head(10)
        

In [ ]:
# extracting the year from date
df['year'] = df['date'].dt.year

In [ ]:
df.head()

In [ ]:
# extracting the month name
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour']  =df['date'].dt.hour

In [ ]:
df['minute']  =df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df[df['users']=="Prerit Sharma"].shape

In [ ]:
# number of words 
words = []
for message in df['message']:
    words.extend(message.split())
    

In [ ]:
len(words)

In [ ]:
# !pip install URLExtract
# COunting the number of links shared in the chat
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("Let's www.gmail.com have URL stackoverflow.com as a example google.com,http://facebook.com,ftp://url.in")
urls



In [ ]:
links = []
for message in df['message']:
    links.extend(extractor.find_urls(message))

In [ ]:
len(links)

In [ ]:
df

In [ ]:
# counting the number of messages per user
x = df['users'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation = "vertical")
plt.show()

In [ ]:
round((df['users'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'users':'Name','count':'Percent'})

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())
    
words
    

In [ ]:
# top 20 words  
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

# Now all these words are basically stop words i.e they are used at the end of sentences
# but dont have a meaning

# soo we need to remove these

# Now we will be removing 
# ->Group Notifications
# ->Stop Words
# ->Media Omitted


In [ ]:
temp = df[df['users'] != "group_notification"]
temp = temp[temp['message'] != '<Media omitted>\n']

In [ ]:
# now since the stop words are in hinglish
# we have imported a file were there are various stop
# words that are in hinglish

f = open('stop_hinglish.txt','r')
stop_words = f.read()
print(stop_words)


In [ ]:
words = []
for message in temp['message']:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)


   

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
%pip install emoji

In [ ]:
# importing emoji library for analysing emojis
import emoji

    

In [ ]:
import emoji

# Initialize an empty list to hold the emojis
emojis = []

# Function to check if a character is an emoji
def is_emoji(character):
    return character in emoji.EMOJI_DATA

# Iterate over each message in the dataframe
for message in df['message']:
    # Extract emojis from each message
    emojis.extend([c for c in message if is_emoji(c)])

# Count the occurrences of each emoji
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))



In [ ]:
# creating a new column and filling it with month number
df['month_num'] = df['date'].dt.month

In [ ]:
df

In [ ]:
# grouping the data on the basis of year and month
timeline = df.groupby(['year','month_num','month']).count()['message'].reset_index()

In [ ]:
timeline

In [ ]:
# creating a new column storing the month and year 

time = []
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i]+"-"+str(timeline['year'][i]))
    
    

In [ ]:
timeline['time'] = time

In [ ]:
timeline
# now this will help in generating a plot where x axis is time and y axis denotes number of messages

In [ ]:
plt.plot(timeline['time'],timeline['message'])
plt.xticks(rotation = "vertical")
plt.show()

In [ ]:
# creating a new column which stores the date only
df['only_date'] = df['date'].dt.date
df

In [ ]:
# grouping the data
daily_timeline = df.groupby('only_date').count()['message'].reset_index()
daily_timeline

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(daily_timeline['only_date'],daily_timeline['message'])

In [ ]:
df['day_name'] = df['date'].dt.day_name()

In [ ]:
df['day_name'].value_counts()

In [ ]:
# heat map

period = []
for hour in df[['day_name','hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

In [ ]:
period

In [ ]:
df.head()

In [ ]:
df["period"] = period

In [ ]:
df

In [ ]:
import seaborn as sns
plt.figure(figsize = (20,6))
# creating a pivot table with x axis as day name , y axis as period , value as number of messages 
# filling the places where there are no messages as 0
sns.heatmap(df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0))
plt.yticks(rotation = "horizontal")
plt.show()

In [ ]:
# pivot table 
df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0)